In [19]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Step 1: Document Collection

In this step, we will simulate a small document corpus (text dataset) consisting of a few short text samples. These documents will be used to build our Inverted Index.

**Talking Point 1:** A good test document collection should contain varied vocabulary and overlapping terms to verify token mapping accuracy in the inverted index.

**Talking Point 2:** For real-world use, text may come from files, web scraping, or databases — but we’ll keep it simple here using plain strings in memory.


In [20]:
from sklearn.datasets import fetch_20newsgroups
import random

# Load a subset of the dataset for quick processing
newsgroups_data = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

# We'll take only the first 20 documents for this workshop
documents = newsgroups_data.data[:20]

# Assign unique IDs to each document
document_dict = {f'doc_{i}': doc for i, doc in enumerate(documents)}

# Preview first document
print(f"Sample doc ID: doc_0\n\n{document_dict['doc_0'][:500]}...")

Sample doc ID: doc_0

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail....


##  Step 2: Tokenizer Implementation

Talking Point 2 (Markdown):
A tokenizer breaks raw text into words. A custom tokenizer gives us flexibility to handle domain-specific patterns, punctuation, and edge cases. For example, we might want to keep hyphenated words or handle contractions differently.


In [21]:
import re

def tokenize(text):
    """
    Tokenizes the input text into words.
    - Lowercases
    - Removes punctuation
    - Splits on whitespace
    """
    # Lowercase
    text = text.lower()
    # Remove punctuation using regex
    text = re.sub(r'[^\w\s]', '', text)
    # Split into tokens
    tokens = text.split()
    return tokens

# Example usage on one document
sample_tokens = tokenize(document_dict['doc_0'])
print(sample_tokens[:20])

['i', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'i', 'saw', 'the', 'other', 'day', 'it', 'was']


## Step 3: Normalization Pipeline
 Text normalization (like removing stop words and stemming) reduces noise and improves IR accuracy. For example, "running", "runs", and "ran" all reduce to "run" — helping the Inverted Index map them to the same concept.

In [22]:
%pip install nltk

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Download required NLTK data
nltk.download('stopwords')

# Initialize stopwords and stemmer
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def normalize(tokens):
    """
    Normalizes a list of tokens by:
    - Removing stop words
    - Applying stemming
    """
    normalized = []
    for token in tokens:
        if token not in stop_words:
            stemmed = stemmer.stem(token)
            normalized.append(stemmed)
    return normalized

# Example: Normalize a sample document
tokens = tokenize(document_dict['doc_0'])
normalized_tokens = normalize(tokens)

print(normalized_tokens[:20])

Note: you may need to restart the kernel to use updated packages.
['wonder', 'anyon', 'could', 'enlighten', 'car', 'saw', 'day', '2door', 'sport', 'car', 'look', 'late', '60', 'earli', '70', 'call', 'bricklin', 'door', 'realli', 'small']



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shiru\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Step 4: Build and Query the Inverted Index
An Inverted Index maps each term to a list of documents it appears in. This makes search extremely efficient, especially at scale. It's the backbone of search engines and information retrieval systems.

In [23]:
#Build the inverted Index
inverted_index = {
    "term1": {"doc_3", "doc_7"},
    "term2": {"doc_1", "doc_4", "doc_9"},
}

In [24]:
from collections import defaultdict

def build_inverted_index(documents):
    """
    Builds an inverted index from the input dictionary of documents.
    """
    index = defaultdict(set)

    for doc_id, text in documents.items():
        tokens = tokenize(text)
        normalized_tokens = normalize(tokens)

        for token in normalized_tokens:
            index[token].add(doc_id)

    return index

# Build and store the inverted index
inverted_index = build_inverted_index(document_dict)

# Preview sample entries
for i, (term, doc_ids) in enumerate(inverted_index.items()):
    print(f"{term}: {sorted(list(doc_ids))}")
    if i == 9:
        break

wonder: ['doc_0', 'doc_2']
anyon: ['doc_0', 'doc_16', 'doc_18']
could: ['doc_0', 'doc_11', 'doc_17', 'doc_2']
enlighten: ['doc_0']
car: ['doc_0', 'doc_17']
saw: ['doc_0']
day: ['doc_0', 'doc_1', 'doc_13']
2door: ['doc_0']
sport: ['doc_0', 'doc_17']
look: ['doc_0', 'doc_13', 'doc_15', 'doc_2']


# Phrase Query Function
We'll implement a basic query function that:

Normalizes input query

Finds documents containing all tokens (AND operation)

In [25]:
def search(query, index):
    """
    Search for documents that contain **all words** in the query.
    """
    query_tokens = normalize(tokenize(query))

    if not query_tokens:
        return []

    # Initialize result with docs containing the first token
    result_docs = index.get(query_tokens[0], set()).copy()

    # Intersect with other tokens' doc sets
    for token in query_tokens[1:]:
        result_docs &= index.get(token, set())

    return sorted(result_docs)

# 🔍 Run 2 Phrase Queries
query1 = "computer science"
query2 = "space nasa mission"

print(f"\nResults for '{query1}': {search(query1, inverted_index)}")
print(f"Results for '{query2}': {search(query2, inverted_index)}")


Results for 'computer science': []
Results for 'space nasa mission': []


In [26]:
print("Query 1 normalized:", normalize(tokenize("computer science")))
print("Query 2 normalized:", normalize(tokenize("space nasa mission")))


Query 1 normalized: ['comput', 'scienc']
Query 2 normalized: ['space', 'nasa', 'mission']


In [27]:
# Query 1: 'comput', 'scienc'
print("Docs with 'comput':", inverted_index.get('comput', []))
print("Docs with 'scienc':", inverted_index.get('scienc', []))

# Query 2: 'space', 'nasa', 'mission'
print("Docs with 'space':", inverted_index.get('space', []))
print("Docs with 'nasa':", inverted_index.get('nasa', []))
print("Docs with 'mission':", inverted_index.get('mission', []))

Docs with 'comput': {'doc_13', 'doc_2'}
Docs with 'scienc': []
Docs with 'space': {'doc_13'}
Docs with 'nasa': []
Docs with 'mission': {'doc_13'}


In [28]:
def search_or(query, index):
    query_tokens = normalize(tokenize(query))
    result_docs = set()
    for token in query_tokens:
        result_docs |= index.get(token, set())
    return sorted(result_docs)

print("OR Search - computer science:", search_or("computer science", inverted_index))
print("OR Search - space nasa mission:", search_or("space nasa mission", inverted_index))

OR Search - computer science: ['doc_13', 'doc_2']
OR Search - space nasa mission: ['doc_13']


## Phrase Query Tests

*Talking Point 5:* Phrase queries using AND logic are strict — all terms must exist in the same document. If even one is missing, the query returns no result. This highlights how token coverage impacts IR accuracy.
```python
# AND Queries
print("Search (AND) - 'space mission':", search("space mission", inverted_index))
print("Search (AND) - 'comput mission':", search("comput mission", inverted_index))

# OR Queries
print("Search (OR) - 'computer science':", search_or("computer science", inverted_index))
print("Search (OR) - 'space nasa mission':", search_or("space nasa mission", inverted_index))

# 5. Stemming – Applied PorterStemmer
Add Stemming Using NLTK’s PorterStemmer
Here’s exactly what to add to your existing normalize() function

In [29]:
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import string

# Download required resources
nltk.download('stopwords')

# Initialize stemmer and stopword list
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

# Normalization function
def normalize(tokens):
    normalized = []
    for token in tokens:
        token = token.lower().strip(string.punctuation)
        if token and token not in stop_words:
            stemmed = stemmer.stem(token)
            normalized.append(stemmed)
    return normalized

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shiru\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
def tokenizer_for_vectorizer(text):
    return normalize(tokenize(text))

In [31]:
tokens = ["computers", "running", "happily", "drives", "mice"]
print("After stemming:", normalize(tokens))

After stemming: ['comput', 'run', 'happili', 'drive', 'mice']


# Step 6: Vectorization and Cosine Similarity
Vectorization converts processed text into numerical form so we can compute similarity between documents. We use TF-IDF to give higher weight to rare but meaningful terms. Cosine similarity helps identify how similar two documents are in terms of their content, regardless of their length.


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Reuse the same documents (we'll only use 5 here for quick testing)
sample_docs = [document_dict[f'doc_{i}'] for i in range(5)]

# Initialize vectorizer with preprocessing
vectorizer = TfidfVectorizer(
    tokenizer=tokenize,         # Custom tokenizer
    stop_words='english',       # Built-in stopword removal
    lowercase=True              # Lowercasing
)

# Transform documents into TF-IDF matrix
tfidf_matrix = vectorizer.fit_transform(sample_docs)

# Compute cosine similarity matrix (doc-to-doc)
cosine_sim_matrix = cosine_similarity(tfidf_matrix)

# Display similarity between doc_0 and others
print("Cosine Similarity between doc_0 and other docs:")
for i in range(len(sample_docs)):
    print(f"doc_0 vs doc_{i}: {cosine_sim_matrix[0][i]:.4f}")

Cosine Similarity between doc_0 and other docs:
doc_0 vs doc_0: 1.0000
doc_0 vs doc_1: 0.0124
doc_0 vs doc_2: 0.0538
doc_0 vs doc_3: 0.0000
doc_0 vs doc_4: 0.0000


e:\Lab6\venv\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [33]:
print("TF-IDF Vocabulary:", list(vectorizer.vocabulary_.keys()))


TF-IDF Vocabulary: ['wondering', 'enlighten', 'car', 'saw', 'day', '2door', 'sports', 'looked', 'late', '60s', 'early', '70s', 'called', 'bricklin', 'doors', 'really', 'small', 'addition', 'bumper', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'engine', 'specs', 'years', 'production', 'history', 'info', 'funky', 'looking', 'email', 'fair', 'number', 'brave', 'souls', 'upgraded', 'si', 'clock', 'oscillator', 'shared', 'experiences', 'poll', 'send', 'brief', 'message', 'detailing', 'procedure', 'speed', 'attained', 'cpu', 'rated', 'add', 'cards', 'adapters', 'heat', 'sinks', 'hour', 'usage', 'floppy', 'disk', 'functionality', '800', '14', 'm', 'floppies', 'especially', 'requested', 'summarizing', 'days', 'network', 'knowledge', 'base', 'upgrade', 'havent', 'answered', 'thanks', 'folks', 'mac', 'plus', 'finally', 'gave', 'ghost', 'weekend', 'starting', 'life', '512k', 'way', '1985', 'sooo', 'im', 'market', 'new', 'machine', 'bit', 'sooner', 'intended', 'picking', 'powerbook', '1

# Query Test (TF-IDF + Cosine):
We now test the similarity of a user query like "space shuttle program" to each of the documents and rank them.

In [34]:
query = ["floppy disk"]
query_vec = vectorizer.transform(query)
similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()

ranked_docs = sorted(zip(range(len(sample_docs)), similarities), key=lambda x: -x[1])

print(f"\nQuery: '{query[0]}'")
print("Ranked documents (doc_id, similarity score):")
for doc_id, score in ranked_docs:
    print(f"doc_{doc_id}: {score:.4f}")



Query: 'floppy disk'
Ranked documents (doc_id, similarity score):
doc_1: 0.1669
doc_2: 0.0714
doc_0: 0.0000
doc_3: 0.0000
doc_4: 0.0000
